## 1. Prepare logbook 

In [1]:
from ipywidgets import widgets
from IPython.display import display

from pprint import pprint
from ipywidgets.widgets import interact

In [2]:
import os, os.path
import datetime

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec

import matplotlib as mpl

In [4]:
font = {'family' : 'monospace',
       'size' : '14'}
mpl.rc('font', **font)

In [45]:
from PIL import Image
import numpy as np
import pandas as pd
import re

## 2. Select Counts vs Time files

In [6]:
default_dir = '/Volumes/My Book Thunderbolt Duo/IPTS/IPTS-16259/Day_4_CO-4_Uptake/time_stamps/'

In [7]:
from PyQt4 import QtGui
%gui qt

In [8]:
try:
    from PySide import QtCore, QtGui
except ImportError:
    from PyQt4 import QtCore, QtGui

def gui_fname(dir=None):
    """Select a directory
    """
    if dir is None: dir ='./'
    dir_name = QtGui.QFileDialog.getOpenFileNames(None, "Select txt files ...",
                                                  dir,
                                                 filter = "ASCII (*.txt);;All (*.*)")
                                                 
    return dir_name

In [11]:
%gui qt
list_files = gui_fname(dir = default_dir)
#pprint(list_files)

## 3. Load the data and metadata 

In [128]:
def retrieve_metadata(file_name):
    """
    return a dictionary of the metadata (plus number of comment lines)
    """
    with open(file_name) as f:
        _metadata = {}
        _nbr_comment_line = 0
        for line in f:
            li = line.strip()
            if li.startswith("#"):
                _nbr_comment_line += 1
                m = re.match(r"^#(?P<tag_name>[,\(\)\s\w]*):(?P<tag_value>.*)", li)
                if m is None:
                    continue
                _metadata[m.group('tag_name')] = m.group('tag_value')
        _metadata['nbr_comment_line'] = _nbr_comment_line
        return _metadata

In [129]:
def retrieve_data(file_name, nbr_comment_line=0, col_names = None):
    file_0 = pd.read_csv(file_name, 
                         skiprows = nbr_comment_line, 
                         header = None,
                         names = col_names)
    return file_0

In [132]:
_metadata = retrieve_metadata(list_files[0])
_col_names = _metadata['Label'].split(',')
_data = retrieve_data(list_files[0], 
                      nbr_comment_line=_metadata['nbr_comment_line'],
                      col_names = _col_names)


In [136]:
w = widgets.IntProgress()
w.max = len(list_files)
display(w)
index = 0

files_data = {}
files_metadata = {}
for index, file in enumerate(list_files):
    _metadata = retrieve_metadata(file)
    files_metadata[file] = _metadata
    _col_names = _metadata['Label'].split(',')
    _data = retrieve_data(file, 
                          nbr_comment_line = _metadata['nbr_comment_line'],
                          col_names = _col_names)
    files_data[file] = _data

    index += 1
    w.value = index